# Dashboard

In [1]:
import random

import numpy as np
import pandas as pd

from rubicon_ml import Rubicon
from rubicon_ml.viz import DataframePlot, ExperimentsTable, MetricCorrelationPlot, MetricListsComparison
from rubicon_ml.viz.dashboard import Dashboard

In [2]:
dates = pd.date_range(start="1/1/2010", end="12/1/2020", freq="MS")

rubicon = Rubicon(persistence="memory", auto_git_enabled=True)
project = rubicon.get_or_create_project("dashboard composition")

for i in range(0, 10):
    experiment = project.log_experiment()

    experiment.log_parameter(name="is_standardized", value=random.choice([True, False]))
    experiment.log_parameter(name="n_estimators", value=random.randrange(2, 10, 2))
    experiment.log_parameter(name="sample", value=random.choice(["A", "B", "C", "D", "E"]))

    experiment.log_metric(name="accuracy", value=random.random())
    experiment.log_metric(name="confidence", value=random.random())

    experiment.log_metric(name="coefficients", value=[random.random() for _ in range(0, 5)])
    experiment.log_metric(name="stderr", value=[random.random() for _ in range(0, 5)])

    data = np.array([list(dates), np.linspace(random.randint(0, 15000), random.randint(0, 15000), len(dates))])
    data_df = pd.DataFrame.from_records(data.T, columns=["calendar month", "open accounts"])

    experiment.log_dataframe(data_df, name="open accounts")

In [ ]:
default_dashbaord = Dashboard(experiments=project.experiments())
default_dashbaord.serve(run_server_kwargs={"port": 8058})

Dash is running on http://127.0.0.1:8058/

 * Serving Flask app 'rubicon_ml.viz.base' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [ ]:
Dashboard(
    experiments=project.experiments(),
    widgets=[
        [
            ExperimentsTable(is_selectable=True),
            MetricCorrelationPlot(selected_metric="accuracy"),
        ],
        [
            
            MetricListsComparison(column_names=[f"var_00{i}" for i in range(0, 5)]),
            DataframePlot(dataframe_name="open accounts"),
        ],
    ],
).serve()